In [1]:
import cudf, cuml, cugraph
import numpy as np

from cuml.preprocessing.model_selection import train_test_split
from cuml.preprocessing.LabelEncoder import LabelEncoder

In [140]:
#extraccion datasets
data_train = cudf.read_csv('dataTitanic/train.csv')
data_test = cudf.read_csv('dataTitanic/test.csv')

In [ ]:
#Verificar toda la informacion de la data train
print('Datos generales del Dataset Train:', data_train.head(10))
print('Verificar dimensión del Dataset Train:', data_train.shape)
print('Verificar valores nulos por cada columna Train:', data_train.isnull().sum())

#Verificar toda la informacion de la data test
print('Datos generales del Dataset Test:', data_test.head())
print('Verificar dimensión del Dataset Test:', data_test.shape)
print('Verificar valores nulos por cada columna Test: ', data_test.isnull().sum())

In [142]:
#Transformation data

#cambiar valores categoricos de la columna sexo a números
data_train['Sex'].replace(['female','male'],[0,1],inplace=True)
data_test['Sex'].replace(['female','male'],[0,1],inplace=True)

#cambiar valores categoricos de la columna enmarked a números
data_train['Embarked'].replace(['Q','S', 'C'],[0,1,2],inplace=True)
data_test['Embarked'].replace(['Q','S', 'C'],[0,1,2],inplace=True)

#calcular la media de los valores faltantes de la columna
#age de train, test y columna fare de test
data_train["Age"].mean()
data_test["Age"].mean()
data_test["Fare"].mean()

promedio_edad = 30
promedio_fare = 35

data_train['Age'] = data_train['Age'].fillna(promedio_edad).to_pandas()
data_test['Age'] = data_train['Age'].fillna(promedio_edad).to_pandas()
data_test['Fare'] = data_train['Fare'].fillna(promedio_fare).to_pandas()

#Eliminar columnas innecesarias, ya que algunas contienen muchos datos faltantes
data_train.drop_column('Cabin')
data_test.drop_column('Cabin')

#eliminar filas con los datos perdidos
data_train.dropna(axis=0, how='any')
data_test.dropna(axis=0, how='any')

data_train['FamilySize'] = data_train['SibSp'] + data_train['Parch'] + 1
data_test['FamilySize'] = data_test['SibSp'] + data_test['Parch'] + 1

#eliminar filas con los datos perdidos nuevamente
data_train.dropna(axis=0, how='any')
data_test.dropna(axis=0, how='any')

data_train.drop_column('Ticket')
data_test.drop_column('Ticket')
data_test.drop_column('PassengerId')
data_train.drop_column('Name')
data_test.drop_column('Name')

In [ ]:
data_train.head()

In [ ]:
data_test.head()

In [ ]:
#Data Splitting with cuml
import cupy as cp
from cuml.preprocessing.model_selection import train_test_split

#Separacion de columnas Sobrevivientes de dataTrain
gdf_dataX = data_train.drop(['Survived'])
gdf_dataY = data_train["Survived"]

#Divide datasets in train and test
#datatrain 80% and datatest 20%
X_train, X_test, y_train, y_test = train_test_split(gdf_dataX, gdf_dataY, train_size=0.8)

#information
print(f'Original datasets: {gdf_dataX.shape[0], gdf_dataY.shape[0]} elements')
print(f'Data X_train: {X_train.shape[0]} elements')
print(f'Data X_test: {X_test.shape[0]} elements')
print(f'Data y_train: {y_train.shape[0]} elements')
print(f'Data y_test: {y_test.shape[0]} elements')

In [ ]:
data_train.dtypes

In [ ]:
import cupy as cp

from cuml.preprocessing.LabelEncoder import LabelEncoder

le = LabelEncoder()
sex = le.fit_transform(X_train.Sex)



#data_Train = X_train.as_gpu_matrix()

In [ ]:
import cupy as cp
from cuml.preprocessing import LabelBinarizer
from cuml import LinearRegression
from cuml.linear_model import LinearRegression

lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "eig")

reg = lr.fit(X_train,y_train)

In [ ]:
#Crear bandas de edades
bandas = [0,8,15,18,25,30,50,80,100]
nombres = ['niños_pequeños','niños_grandes','adolecente','adulto','adultogrande','anciano']

#Categoria par train y test
data_train['Age'] = cudf.Series(data_train['Age'], dtype='category')
data_test['Age'] = cudf.Series(data_test['Age'], dtype='category')

#Agregar las nuevas categorias a train y test
data_train['Age'].cat.add_categories(bandas, labels=nombres)
data_test['Age'].cat.add_categories(bandas, labels=nombres)

In [ ]:
data_trainAge = data_train['Age'].to_frame()
data_testAge = data_test['Age'].to_frame()

In [ ]:
data_trainNuevo = data_train.merge([data_train, data_train.drop(data_train['Age'])], axis=1)